In [2]:
import pandas as pd
import datetime as dt
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objs as go
from plotly.subplots import make_subplots
# Set notebook mode to work in offline
#pyo.init_notebook_mode()

from functions import string_datediff, sunburst_data

pd.options.mode.chained_assignment = None  # default='warn'

In [3]:
beds_ab = pd.read_csv('data-ingest/data/bed_use_powys.csv', parse_dates = [0])
beds_ab.head()

,date,indicator,count
0,2020-04-01,CO_GABed_C19_Con,44
1,2020-04-01,CO_GABed_C19_Sus,67
2,2020-04-01,CO_GABed_NonC19,486
3,2020-04-01,CO_GABed_Spare,353
4,2020-04-01,CO_IVBed_C19_Con,15


In [4]:
lockdowns = pd.read_csv('data-ingest/data/lockdown_status_p.csv', parse_dates = [1]).drop(columns = ['health_board'])
lockdowns

,date,hb_pop,num_lockdown,prop_lockdown
0,2020-10-23,132435,132435,2


In [8]:
colors = ['#747875', '#50869D', '#50869D', '#ED8103', '#ED8103']
statuses = ['(?)',  'IV_NonC19', 'GA_NonC19', 'IV_C19', 'GA_C19']

startdate = "2020-10-23"

fig = make_subplots(rows=1, cols=3, subplot_titles = ("Three weeks before", "First day", "21st day"))

for day,dom in zip([string_datediff(startdate, x) for x in [-21, 0, 21]], [[0, 0.33], [0.33, 0.66], [0.66, 1]]):
    data = px.sunburst(sunburst_data(beds_ab, day), 
                       path = ['bed_type', 'bed_status', 'indicator'], 
                       values = 'count', color = 'bed_status',
                       color_discrete_map = {stat: col for stat, col in zip(statuses, colors)}
    )._data
    trace = go.Sunburst(
        labels=data[0]['labels'],
        parents=[x.split('/')[1] if len(x.split('/')) > 1 else x for x in data[0]['parents'].tolist()],
        values=data[0]['values'],
        domain={'x': dom, 'y': [0.2, 0.95]},
        marker = dict(colors = data[0]['marker']['colors']),
        branchvalues = 'total',
        insidetextorientation='horizontal'
    )
    fig.add_trace(trace) 

fig.update_layout(height=600, width=1000,
                  title_text="Bed use in Powys Health Board during and after firebreak lockdown period",
                 #uniformtext=dict(minsize=10, mode='hide')
                 )

labcolors = ['#50869D', '#ED8103', '#747875']
for i, lab in enumerate(['Spare beds / Other patients', 'COVID-19 patients', 'IV = "Invasive / Ventilated";\t\tGA = "General / Acute"']):
    fig.add_annotation(dict(font=dict(color=labcolors[i],size=12),
                                        x=0,
                                        y= 0+i/20,
                                        showarrow=False,
                                        text=lab,
                                        textangle=0,
                                        xanchor='left',
                                        xref="paper",
                                        yref="paper"))
    
fig.write_html('visualisations/AB_beds_firebreak.html')
fig.show()

In [24]:
beds_ab.tail()

,date,indicator,count
2179,2020-11-18,CO_IVBed_C19_Con,7
2180,2020-11-18,CO_IVBed_C19_Rec,1
2181,2020-11-18,CO_IVBed_C19_Sus,6
2182,2020-11-18,CO_IVBed_NonC19,3
2183,2020-11-18,CO_IVBed_Spare,7


In [16]:
## NEXT STEPS
## Do the same for ~ three weeks after lockdown
## Repeat the whole notebook for a hb that entered lockdown later - compare first day and three weeks later
# can we add sus / con / rec as children of C19?

LOOK AT RECOVERING PATIENTS WITH ADMISSIONS AND DE-ADMISSIONS